In [1]:
from pymongo import MongoClient

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [28]:
# companies = list(coll.find()) # 18801

# La query en compass es: {$or:[{"offices.latitude":{$ne:null}},{"offices.longitude":{$ne:null}}]}

companies = list(coll.find({'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]})) # 14082

In [29]:
len(companies) # 14082

14082

In [32]:
companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [40]:
companies_data = []
for i in range(len(companies)):
    aux_dict = {}
    aux_dict['_id'] = companies[i]['_id']
    aux_dict['name'] = companies[i]['name']
    aux_dict['founded_year'] = companies[i]['founded_year']
    aux_dict['category_code'] = companies[i]['category_code']
    aux_dict['total_money_raised'] = companies[i]['total_money_raised']
    aux_dict['deadpooled_year'] = companies[i]['deadpooled_year']        
    # aux_dict['latitude'] = companies[i]['offices'][0]['latitude']
    # aux_dict['longitude'] = companies[i]['offices'][0]['longitude']
    if len(companies[i]['offices'])>0:
        aux_offices = []
        for j in range(len(companies[i]['offices'])):
            aux_offices.append([j+1, companies[i]['offices'][j]['latitude'],
                                companies[i]['offices'][j]['longitude']])
        aux_dict['offices'] = aux_offices
    companies_data.append(aux_dict)

In [41]:
# En database crear una columna por oficina

In [42]:
# ?
# city
# country_code

In [43]:
len(companies_data) # 14082

14082

In [50]:
companies_data[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'founded_year': 1996,
 'category_code': 'enterprise',
 'total_money_raised': '$0',
 'deadpooled_year': 2,
 'offices': [[1, 37.692934, -121.904945]]}

In [52]:
# Solo la primera oficina de cada compañía:
def getLocation(company):
    longitude = company['offices'][0][2]
    latitude = company['offices'][0][1]
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

getLocation(companies_data[0])

# for e in companies_data:
    # getLocation(companies_data[e])

{'type': 'Point', 'coordinates': [-121.904945, 37.692934]}

In [ ]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
# 30% of the company have at least 1 child.
# Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
# Account managers need to travel a lot
# All people in the company have between 25 and 40 years, give them some place to go to party.
# The CEO is Vegan

In [ ]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

# {founded_year:{$gt: 2009}} # 146
# {founded_year:{$lte: 2009}} # 12990 (no quieres tenerlas cerca)
# Año en que quebraron

# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
# import geopy.distance
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# print geopy.distance.vincenty(coords_1, coords_2).km

# Al parecer hay tambien un operador near en mongodb (lo tenemos en los apuntes,
# poner primero el field de la manera que lo quier Mongo)

# Tabla limpia a mongo Db para hacer querys?

In [ ]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# category_code
# founded_year
# total_money_raised (o dentro de las rondas: funding_rounds -> raised_amount & raised_currency_code)
# ipo.valuation_amount # no vale para startups -> Dinero por el que salen a bolsa
# Año en que quebraron, porque si quebraron no hay que tenerlas en cuenta